# Hill Cipher

Hill cipher is a cipher technique to encrypt plain text using key as a matrix.
Assume the key matrix is K and we need to encrypt plain text P, then cipher text C would be -
C = P x K
We must make sure that dimensions of P and K are compatible for multiplication and inverse of K should exist.

We would take the key K's dimension as 4x4. So, dimension of C and P must be n x 4.

## Implementation

Firstly we define an encoding function that will be used to take plaintext and convert it to a 26 character encoding. (By converting all letters to upper case and discarding all remaining characters). We also add padding character Z to the end of the string to make the length a multiple of 4 (if needed)

Also, let M = 4 be the dimension of the square key matrix.

In [122]:
M = 4

In [123]:
def encode(string):
    result = ''
    for letter in string:
        if letter.isalpha():
            result += letter.upper()
    padding_length = (M - len(result)) % M
    return result + 'Z' * padding_length

def decode(string):
    return string.rstrip('Z')

Lets declare a plain text that we would need to encrypt.

In [124]:
P = 'This is another plain text.'

Encoding this string, we get -

In [125]:
T = encode(P)
print(T)

THISISANOTHERPLAINTEXTZZ


Since we will be only having 26 characters, we declare Zp as closed ring of 26 integers.

In [126]:
Zp = Integers(26)
print(Zp)

Ring of integers modulo 26


We now declare a key matrix for the encryption.

In [127]:
elements = [[randint(0, 25) for _ in range(M)] for __ in range(M)]
key = matrix(Zp, M, M, elements)
while True:
    elements = [[randint(0, 25) for _ in range(M)] for __ in range(M)]
    key = matrix(Zp, M, M, elements)
    try:
        key.inverse()
        break
    except Exception as e:
        continue
print(key)

[ 9  8  7 13]
[25  1 10 22]
[ 1 12 18  7]
[23  5  0  9]


Using the definition of hill cipher, we can encrypt each character of text using -

C = P x K

We must also add padding characters 'Z' at the end of the plain text to make the length a multiple of M(=4) and then convert the text into a matrix.

In [128]:
def hillcipher(text, cipher_key):
    cipher = ''
    text_elements = []
    for i in range(0, len(text), M):
        text_elements.append([ord(x) - ord('A') for x in list(text[i:i+M])])
    
    text_matrix = matrix(Zp, len(text) / M, M, text_elements)
    cipher_matrix = text_matrix * cipher_key
    
    for row in cipher_matrix:
        for e in row:
            cipher += chr(int(Zp(e)) + ord('A'))
    
    return cipher
    

def hilldecipher(cipher_text, cipher_key):
    text = ''
    
    cipher_elements = []
    
    for i in range(0, len(cipher_text), M):
        cipher_elements.append([ord(x) - ord('A') for x in list(cipher_text[i:i+M])])
    
    cipher_matrix = matrix(Zp, len(cipher_text)/M, M, cipher_elements)
    
    text_matrix = cipher_matrix * cipher_key.inverse()
    
    for row in text_matrix:
        for e in row:
            text += chr(int(Zp(e)) + ord('A'))
    # remove any trailing Z's
    # they are padding characters
    return text

Now we test the cipher and decipher algorithms by encrypting and decrypting the text P

In [135]:
T = encode(P)
C = hillcipher(T, key)
D = hilldecipher(C, key)
D_stripped = decode(D)
print(f'Given text - "{P}"')
print(f'Encoded - {T}')
print(f'Key -\n{key}')
print(f'Cipher text - {C}')
print(f'Decipher text - {D}')
print(f'Stripped text - {D_stripped}')

Given text - "This is another plain text."
Encoded - THISISANOTHERPLAINTEXTZZ
Key -
[ 9  8  7 13]
[25  1 10 22]
[ 1 12 18  7]
[23  5  0  9]
Cipher text - OHJVPRCTYBYJTXZEONINIEVZ
Decipher text - THISISANOTHERPLAINTEXTZZ
Stripped text - THISISANOTHERPLAINTEXT


## Test Against Builtin Cipher

Now, we can test the result against the built in Hill Cipher in sagemath.

In [142]:
A = HillCryptosystem(AlphabeticStrings(), 4)
E = A.encoding(encode(P))
print(f'Text - {P}')
print(f'Encoded - {E}')
print(f'Key -\b{key}')
C_test = A.enciphering(key, E)
D_test = A.deciphering(key, C_test)

# convert to python string
C_test = str(C_test)
D_test = str(D_test)
D_test_stripped = decode(D_test)

print(f'Cipher text - {C_test}')
print(f'Decipher text - {D_test}')
print(f'Stripped text - {D_test_stripped}')

Text - This is another plain text.
Encoded - THISISANOTHERPLAINTEXTZZ
Key -[ 9  8  7 13]
[25  1 10 22]
[ 1 12 18  7]
[23  5  0  9]
Cipher text - OHJVPRCTYBYJTXZEONINIEVZ
Decipher text - THISISANOTHERPLAINTEXTZZ
Stripped text - THISISANOTHERPLAINTEXT


Comparing the built in cipher result with our implementation -

In [143]:
print('Results \t Implementation \t Built-in\n')
print(f'Cipher Text \t {C} \t {C_test}')
print(f'Decipher Text \t {D} \t {D_test}\n')
if C_test == C and D_test == D:
    print('Implementation is CORRECT')
else:
    print('Implementatiokn is INCORRECT')

Results 	 Implementation 	 Built-in

Cipher Text 	 OHJVPRCTYBYJTXZEONINIEVZ 	 OHJVPRCTYBYJTXZEONINIEVZ
Decipher Text 	 THISISANOTHERPLAINTEXTZZ 	 THISISANOTHERPLAINTEXTZZ

Implementation is CORRECT


## Cryptoanalysis